# Inference & Evaluation for Our Model

## Importing Data & Libraries

In [1]:
! pip install --upgrade scipy networkx
! pip3 install nltk
! pip install --upgrade scipy
! pip install rouge
!pip install -U sentence-transformers

  Using cached scipy-1.10.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.5 MB)
  Using cached networkx-3.0-py3-none-any.whl (2.0 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.9.3
    Uninstalling scipy-1.9.3:
      Successfully uninstalled scipy-1.9.3
  Attempting uninstall: networkx
    Found existing installation: networkx 2.8.6
    Uninstalling networkx-2.8.6:
      Successfully uninstalled networkx-2.8.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.0.0 requires scipy<1.10,>=1.4.1, but you have scipy 1.10.1 which is incompatible.
     |████████████████████████████████| 1.5 MB 7.3 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 13.4 MB/s  eta 0:00:01
     |████████████████████████████████| 85 kB 3.6 MB/s eta 0:00:011
     |████████████████████████████████| 24.2 MB 6.2 MB/s eta 0:

In [1]:
import scipy
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import nltk
import json
from queue import Queue
import matplotlib.pyplot as mp
from sentence_transformers import SentenceTransformer
from statistics import mean
from bs4 import BeautifulSoup
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/sarchakr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
g = pickle.load(open('./cg_kg.gpickle', 'rb'))

In [3]:
cso_df = pd.read_csv("CSO_data/CSO_entities_ensembled_Sign_bart-large.csv", index_col=0)
cso_df.head()

,cso_number,symptom,root_cause,remediations,description,short_description,affected_services,child_cso,short_term_fix,customer_impacts
0,17536,The Adobe Sign Customized Email Template (CEMT...,Sign Business Intelligence team implemented a ...,There are none.,Adobe Sign Customized Email Template users are...,Adobe Sign - Customized Email Template microse...,"316391, 320691",NaN,The BI change was rolled back and APO team cle...,Customized Email Template (CEMT) microservice ...
1,17522,Starting on 2022-05-24 between 11:29 UTC and 1...,canary deployment.,There are none.,Incident description: Cloudwatch check for 5xx...,Identity Management Services,"318561, 318560, 318205, 321727, 320998, 321962...",NaN,There will be a rollback of the rollback.,downstream services could have experience re-...
2,17510,Microsoft Office365 Provider microservice expo...,The problem was caused by a misstep in convert...,Review the manual testing procedure with team ...,Microsoft Office365 Provider unable to run com...,Microsoft Office365 Provider,317501,NaN,the data structure that is called.,Some commands in the Microsoft Office365 Provi...
3,17401,Some Adobe Sign users might have been unable t...,Some North America Adobe Sign users might have...,Review the SOP for DB nodes upgrades. Validate...,Incident description: Adobe Sign NA2 Compose P...,Adobe Sign North America 2,"321671, 323232, 321670, 323231, 323230, 317791...",17402,was set as primary node.,Some North America Adobe Sign users might have...
4,17368,The PDFcombiner worker for Sign North America ...,The Sign service operates as a distributed sys...,There is an underlying bug in JMS message hand...,Agreements are getting stuck,Adobe Sign - Adobe Sign North America 3,"322299, 318358",NaN,instances were restarted.,3900 requests expired during the impact window...


In [4]:
alert_df = pd.read_csv("./Alerts_data/alert_id_mapping.csv", index_col=0)
alert_df.head()

,alert_name,id
0,quartz_scheduler_down_alert,0
1,qpid shardau1toeu1q queuesize,1
2,Splunk Alert: TechOps - JMS Send Failure,2
3,app server load5 is high,3
4,Pingdom check Adobe Sign DNS Test,4


In [5]:
# Get the important information from the alerts and the outage like started time, symptom etc.

df_alerts_sorted = pd.read_csv('./Alerts_data/all_alerts_parsed.csv', index_col=0)
df_alerts_sorted['created_at'] = pd.to_datetime(df_alerts_sorted['created_at'])
df_alerts_sorted = df_alerts_sorted.sort_values(by = 'created_at')
timelist_alerts = list(df_alerts_sorted['created_at'])
numberlist_alerts = list(df_alerts_sorted['incident_number'])

with open('./CSO_data/CSO_all_scraped_Sign.json', 'r') as f:
    data = json.load(f)

cso_data = {'cso_number':[], 'start_time':[], 'end_time':[]}
for cso, value in data.items():
    cso_data['cso_number'].append(cso)
    cso_data['start_time'].append(value['primaryIncident']['u_cso_started'])
    cso_data['end_time'].append(value['primaryIncident']['u_cso_ended'])


In [6]:
cso_time_df = pd.DataFrame(cso_data)

In [7]:
cg = pickle.load(open('./causal_graph_data/cg_unweighted_filtered.gpickle', 'rb'))
cg_alert_id = nx.get_node_attributes(cg, "alert_id")
alert_ids = [v for k,v in cg_alert_id.items()]

In [8]:
json_file = data

In [9]:
alert_names = df_alerts_sorted.copy()

In [10]:
cso_descr = pd.read_csv("./CSO_data/cso_alert.csv")

76

### Preprocessing some data (specific to our dataset)

In [12]:
def slicer(my_str,sub):
    index=my_str.find(sub)
    if index !=-1 :
        return my_str[index+8:] 
    else :
        raise Exception(my_str,' -----Sub string not found!')

In [13]:
def html_stripper(code):
    return BeautifulSoup(code).get_text().replace('\r',' ').replace('\xa0',' ').replace('\n',' ')

In [14]:
## Removing region name from outage-alert description
cso_descr_data = []
for index, row in cso_descr.iterrows():
    z = row['cso_number']
    x = row['description']
    try:
        y = slicer(x,"<br><br>")
        cso_descr_data.append([z,y])
    except:
        continue
cso_descr_df = pd.DataFrame(cso_descr_data, columns=['cso_number', 'descr'])
len(cso_descr_df)

74

## Functions to Retrieve Required Information

In [16]:
inc_dict = {}
for i, inc in enumerate(numberlist_alerts):
    inc_dict[str(inc)] = i

In [17]:
# FInd the alerts that are inked to an outage
def get_alerts_by_cso(cso, n=3):
    timestr = cso_time_df[cso_time_df['cso_number'] == cso]['start_time'].iloc[0]   # gets cso timestr
    timeend = cso_time_df[cso_time_df['cso_number'] == cso]['end_time'].iloc[0]
    
    timestr = pd.Timestamp(timestr, tz="UTC")
    timeend = pd.Timestamp(timeend, tz="UTC")

    if timestr < timelist_alerts[0]:
        timestr = timelist_alerts[0]
    
    if timeend < timelist_alerts[0]:
        timeend = timelist_alerts[0]
        
    alerts_timeframe = get_alerts_by_time(timestr, timeend, n)
    new_alerts_timeframe = [df_alerts_sorted['alert_id'].iloc[inc_dict[str(alert)]] for alert in alerts_timeframe]
    
    return new_alerts_timeframe
    

In [18]:
# FInd the alerts that occured within a time period
def get_alerts_by_time(start_time, end_time, num):                                           # the input time and the number of different recorded past alert times, we want alerts from
    lst = []
    low = 0
    mx = len(timelist_alerts)
    high = mx
    while(low<high):
        if(timelist_alerts[int((low+high)/2)] > start_time):
            high = int((low+high)/2)
        else:
            low = int((low+high)/2)+1
    ind = low-1
    
    j = 0
    k = 0
    
    while(j < low-1 and (start_time - timelist_alerts[low-1-j]) < pd.Timedelta(hours=1)):
        lst.append(numberlist_alerts[low - 1 -j])
        while(j<low-1 and timelist_alerts[low-1-j] == timelist_alerts[low-2-j]):
            lst.append(numberlist_alerts[low-2-j])
            j = j + 1
        j = j + 1
        k = k + 1
    j = low
    k = 0
    while(j<mx and (end_time - timelist_alerts[j]) > pd.Timedelta(minutes=1)):
        lst.append(numberlist_alerts[j])
        while(j<mx-1 and timelist_alerts[j] == timelist_alerts[j+1]):
            lst.append(numberlist_alerts[j+1])
            j = j + 1
        j = j + 1
        k = k + 1
    return list(set(lst))

In [19]:
def get_top_n_rc(cso_dict, n):
    return dict(sorted(cso_dict.items(), key=lambda x: x[1], reverse=True)[:n])

In [20]:
def get_alert_name_from_id(alert_id):
    temp = alert_df.index[alert_df['id']==alert_id].tolist()
    for i in temp:
        return alert_df['alert_name'][i]

In [21]:
def get_sym_description(cso_number):
    temp = cso_df.index[cso_df['cso_number']==int(cso_number)].tolist()[0]
    return cso_df['symptom'][temp]

In [22]:
def get_rc_description(cso_number):
    temp = cso_df.index[cso_df['cso_number']==int(cso_number)].tolist()[0]
    return cso_df['root_cause'][temp]

In [23]:
def get_rem_description(cso_number):
    temp = cso_df.index[cso_df['cso_number']==int(cso_number)].tolist()[0]
    return cso_df['remediations'][temp]

## Inference Functions

### Pred: path-based inference method

In [24]:
## path_based_inference
def path_based_inf(alert_id, k, g, q, visited_dict):
    alert_id_cg = {value:key for key,value in cg_alert_id.items()}
    
    rc = {}
    while(not q.empty()):
        (node, k_) = q.get()
        if k_ > 0:            
            for (u, v, dic) in g.out_edges(alert_id_cg[node], data = True):
                if (not visited_dict[v]):
                    if (dic['label'] == 'caused by') or (dic['label'] == 'causes'):
                        visited_dict[v] = True
                        q.put((cg_alert_id[v], k_ - 1))
                    elif (dic['label'] == 'caused CSO') or (dic['label'] == 'alert-symptom link'):
                        if v[:-4] in rc.keys():
                            rc[v[:-4]] += 1/(k-k_+1)
                        else:
                            rc[v[:-4]] = 1/(k-k_+1)
                
    return rc

In [25]:
def find_rc_for_set(alert_ids,k,g):
    rc = {}
    visited_dict = {}
    for node in g.nodes():
        visited_dict[node] = False
    q = Queue()
    if(isinstance(alert_ids,list)):
        for alert_id in alert_ids:
            visited_dict[alert_id] = True
            q.put((alert_id,k))
            temp = path_based_inf(alert_id, k, g, q, visited_dict)
            for foo in temp.keys():
                if foo in rc.keys():
                    rc[foo] += temp[foo]
                else:
                    rc[foo] = temp[foo]
        for node in g.nodes():
            visited_dict[node] = False
    else:
        visited_dict[alert_ids] = True
        q.put((alert_ids,k))
        rc = find_rc_single_alert_vis(alert_ids, k, g, q, visited_dict)
    foo= dict(sorted(rc.items(), key=lambda x: x[1], reverse=True)[::])
    poo = {}
    for i in foo.keys():
        poo[int(i)] = foo[i]
    return poo

In [26]:
def find_rc_for_set_normalised(alert_ids,k,g):
    rc = {}
    visited_dict = {}
    for node in g.nodes():
        visited_dict[node] = False
    q = Queue()
    
    for alert_id in alert_ids:
        visited_dict[alert_id] = True
        q.put((alert_id,k))
        temp = path_based_inf(alert_id, k, g, q, visited_dict)
        for foo in temp.keys():
            if foo in rc.keys():
                rc[foo] += temp[foo]
            else:
                rc[foo] = temp[foo]
    for node in g.nodes():
        visited_dict[node] = False
    
    foo= dict(sorted(rc.items(), key=lambda x: x[1], reverse=True)[::])
    norm_factor = sum(list(foo.values()))
    poo = {}
    for i in foo.keys():
        poo[int(i)] = foo[i]/norm_factor
    return poo

### Sim: Similarity-based Inference Method

In [190]:
from transformers import BertTokenizer, BertModel
import torch

model_bert = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True,)
tokenizer_bert = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [191]:
def bert_text_preparation(text, tokenizer):
    """Preparing the input for BERT
    
    Takes a string argument and performs
    pre-processing like adding special tokens,
    tokenization, tokens to ids, and tokens to
    segment ids. All tokens are mapped to seg-
    ment id = 1.
    
    Args:
        text (str): Text to be converted
        tokenizer (obj): Tokenizer object
            to convert text into BERT-re-
            adable tokens and ids
        
    Returns:
        list: List of BERT-readable tokens
        obj: Torch tensor with token ids
        obj: Torch tensor segment ids
    
    
    """
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    return tokenized_text, tokens_tensor, segments_tensors

In [192]:
def get_bert_embeddings(tokens_tensor, segments_tensors, model):
    """Get embeddings from an embedding model
    
    Args:
        tokens_tensor (obj): Torch tensor size [n_tokens]
            with token ids for each token in text
        segments_tensors (obj): Torch tensor size [n_tokens]
            with segment ids for each token in text
        model (obj): Embedding model to generate embeddings
            from token and segment ids
    
    Returns:
        list: List of list of floats of size
            [n_tokens, n_embedding_dimensions]
            containing embeddings for each token
    
    """
    
    # Gradient calculation id disabled
    # Model is in inference mode
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # Removing the first hidden state
        # The first state is the input state
        hidden_states = outputs[2][1:]

    # Getting embeddings from the final BERT layer
    token_embeddings = hidden_states[-1]
    # Collapsing the tensor into 1-dimension
    token_embeddings = torch.squeeze(token_embeddings, dim=0)
    # Converting torchtensors to lists
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]

    return list_token_embeddings

In [204]:
def get_symptom_embeddings():
    symptom_list = cso_df['symptom'].to_list()
    
    embeddings = []
    for i in range(len(symptom_list)):
        tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(symptom_list[i], tokenizer_bert)
        list_token_embeddings = np.array(get_bert_embeddings(tokens_tensor, segments_tensors, model_bert))
        sentence_embed = np.mean(list_token_embeddings, axis=0)
        embeddings.append(sentence_embed)
    
    return embeddings

In [205]:
def get_similar_cso_from_description_bert(description, embeddings):
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(description, tokenizer_bert)
    list_token_embeddings = np.array(get_bert_embeddings(tokens_tensor, segments_tensors, model_bert))
    sentence_embed = np.mean(list_token_embeddings, axis=0)
    
    similarity_dict = {}
    for i in range(len(embeddings)):
        similarity_dict[int(cso_df.iloc[i]['cso_number'])] = cosine(sentence_embed, embeddings[i])
    return similarity_dict

In [206]:
def get_similar_cso_from_description_bert_with_embed(cso_embed, embeddings):    
    similarity_dict = {}
    for i in range(len(embeddings)):
        similarity_dict[int(cso_df.iloc[i]['cso_number'])] = cosine(cso_embed, embeddings[i])
    return similarity_dict

In [209]:
def cso_similarity_bert(cso1, emb2):
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(cso1, tokenizer_bert)
    list_token_embeddings = np.array(get_bert_embeddings(tokens_tensor, segments_tensors, model_bert))
    emb1 = np.mean(list_token_embeddings, axis=0)
    
    return cosine(emb1, emb2)

### Clust: Cluster based inference method
### Merge CG and KG graph with KG clusters into effect

In [36]:
def find_symptom_cluster_for_set(alert_ids, k, g, cso_cluster_map):
    cluster = {}
    visited_dict = {}
    for node in g.nodes():
        visited_dict[node] = False
        
    q = Queue()
    if not isinstance(alert_ids, list):
        new_alert_ids = [alert_ids]
    else:
        new_alert_ids = alert_ids
        
    for alert_id in new_alert_ids:
        visited_dict[alert_id] = True
        q.put((alert_id, k))
        temp = path_based_inf(alert_id, k, g, q, visited_dict)

        for foo in temp.keys():
            cl = cso_cluster_map[foo]
            if cl in cluster:
                cluster[cl] += 1
            else:
                cluster[cl] = 1

    for node in g.nodes():
        visited_dict[node] = False
                    
    foo= dict(sorted(cluster.items(), key=lambda x: x[1], reverse=True)[::])
    return foo

In [37]:
def create_rf_inference_df(alert_ids, alert_id_map):
    reverse_map = {v:k for k,v in alert_id_map.items()}
    all_alerts = list(reverse_map.keys())
    
    l = np.zeros(shape=len(all_alerts))
    
    for alert in alert_ids:
        l[reverse_map[alert]] = 1
    
    df = pd.DataFrame([l], columns=all_alerts)
    return df

In [38]:
def rank_cluster(clusters, n_clusters, rf_model, X):
    y_pred_proba = rf_model.predict_proba(X).reshape(-1)
    
    rf_clusters = {float(i):y_pred_proba[i] for i in range(len(y_pred_proba))}
    cluster_rank = dict(sorted(rf_clusters.items(), key=lambda x: x[1], reverse=True)[::])
    
    return cluster_rank

In [226]:
def find_rc_set_bert(clusters, K, cso_cluster_map, cso_embedding):
    cluster_cso_map = {v:k for k,v in cso_cluster_map.items()}
    
    combined_pred = {}
    count = 0
    for cl, _ in sorted(clusters.items(), reverse=True):
        all_csos = [v for k,v in cluster_cso_map.items() if k == cl]
        for i in all_csos:
            combined_pred[i] = cso_similarity_bert(cso_df[cso_df['cso_number'] == int(i)]['symptom'].item(), cso_embedding)
        count += 1
    
    return combined_pred

In [41]:
# Debugging code to check whether Clust inference method is working or not

'''
node_cluster_map = nx.get_node_attributes(g, "cluster_id")
cso_cluster_map = {k.split('_')[0]:v for k,v in node_cluster_map.items()}
n_clusters = len(set([v for v in cso_cluster_map.values()]))
rf = pickle.load(open('random_forest.pickle', 'rb'))

i=14757

df = create_rf_inference_df(get_alerts_by_cso(str(i)), nx.get_node_attributes(g, "alert_id"))

clust1 = find_symptom_cluster_for_set(get_alerts_by_cso(str(i)), 10, g, cso_cluster_map)
clust2 = rank_cluster(clust1, n_clusters, rf, df)

s = sum([1.0/v for k,v in clust1.items()])
clust1_1 = {k: 1.0/(v*s) for k,v in clust1.items()}

clust = {}
for i in range(n_clusters):
    cl = float(i)
    r1 = clust1_1[cl] if cl in clust1_1 else 0
    r2 = clust2[cl] if cl in clust2 else 0
    clust[cl] = r1+r2
# find_rc_set(clust2, 1, cso_cluster_map, csos_descr[i])
'''

## Evaluation Functions

In [40]:
def remove_cso_from_dict(cso_dict1, exception_cso_number):
    cso_dict = cso_dict1.copy()
    if int(exception_cso_number) in cso_dict.keys():
        del cso_dict[int(exception_cso_number)]
    if str(exception_cso_number) in cso_dict.keys():
        del cso_dict[str(exception_cso_number)]
    return cso_dict

### ROUGE Score 

In [41]:
from rouge import Rouge
rouge = Rouge()

In [91]:
# Take max rouge among top-n RCs
def max_rouge(cso_dict, actual_rc, actual_rem, type_rpf, n):
    max_rc = 0
    max_rem = 0
    rc_cso = 0
    rem_cso = 0
    top_dic = get_top_n_rc(cso_dict,n)
    for i in top_dic.keys():
        foo = cso_df.iloc[cso_df.index[cso_df['cso_number']==int(i)].tolist()[0]]
        trc = rouge.get_scores(str(foo['root_cause']), str(actual_rc))[0]['rouge-1'][type_rpf]
        if trc>=max_rc:
            rc_cso = i
            max_rc = trc
        trm = rouge.get_scores(str(foo['remediations']), str(actual_rem))[0]['rouge-1'][type_rpf]
        if trm>= max_rem:
            rem_cso = i
            max_rem = trm
    return max_rc, max_rem, rc_cso, rem_cso

In [271]:
# Take max rouge among top-n RCs by removing the case where prediction == inout outage
def max_rouge_minus1(cso_dict, actual_rc, actual_rem, type_rpf, n):
    max_rc = 0
    max_rem = 0
    rc_cso = 0
    rem_cso = 0
    top_dic = get_top_n_rc(cso_dict,n)
    for i in top_dic.keys():
        foo = cso_df.iloc[cso_df.index[cso_df['cso_number']==int(i)].tolist()[0]]
        trc = rouge.get_scores(str(foo['root_cause']), str(actual_rc))[0]['rouge-1'][type_rpf]
        if trc<1 and trc>=max_rc:
            rc_cso = i
            max_rc = trc
        trm = rouge.get_scores(str(foo['remediations']), str(actual_rem))[0]['rouge-1'][type_rpf]
        if trm<1 and trm>= max_rem:
            rem_cso = i
            max_rem = trm
    return max_rc, max_rem, rc_cso, rem_cso

### Oracle Calculation

In [121]:
def get_oracle_score(auto_cso_list, type_rpf):
    results_rc = {}
    results_rem = {}
    for i in auto_cso_list:
        foo = cso_df.iloc[cso_df.index[cso_df['cso_number']==int(i)].tolist()[0]]
        if type(foo['root_cause'])==float or type(foo['remediations'])==float :
            print(i)
        actual_rc = str(foo['root_cause'])
        actual_rem = str(foo['remediations'])
        series_rc = {}
        series_rem = {}
        for index, row in cso_df.iterrows():
            rc_ro = rouge.get_scores(str(row['root_cause']), actual_rc)[0]['rouge-1'][type_rpf]
            rem_ro = rouge.get_scores(str(row['remediations']), actual_rem)[0]['rouge-1'][type_rpf]
            if rc_ro <1:
                series_rc[row['cso_number']] = rc_ro
            if rem_ro <1:
                series_rem[row['cso_number']] = rem_ro
        #print(i," completed")
        results_rc[i] = dict(sorted(series_rc.items(), key=lambda x: x[1], reverse=True)[::])
        results_rem[i] = dict(sorted(series_rem.items(), key=lambda x: x[1], reverse=True)[::])
    return results_rc, results_rem

## Experiment

In [47]:
## generating random (and valid) CSOs
random_cso_list = []
n = 50 #number of random CSOs needed
while True:
    x = list(cso_time_df.sample()['cso_number'])[0]
    timestr = cso_time_df[cso_time_df['cso_number'] == x]['start_time'].iloc[0]
    timestr = pd.Timestamp(timestr, tz="UTC")
    if len(get_alerts_by_cso(x)) == 0:
        continue
    #print("C1 ",x)
    cond3 = type(list(cso_df[cso_df['cso_number'] == int(x)]['root_cause'])[0]) is str
    cond4 = type(list(cso_df[cso_df['cso_number'] == int(x)]['remediations'])[0]) is str
    if x not in random_cso_list:
        if cond3 and cond4:
            random_cso_list.append(int(x))
        #print("C2 ",x)
    if len(random_cso_list)==n:
        #print("C3 ",x)
        break


[6894,
 7836,
 14055,
 10336,
 14965,
 9206,
 7500,
 6578,
 17401,
 14224,
 6326,
 14377,
 12088,
 7762,
 15560,
 6323,
 11993,
 9131,
 10999,
 7555,
 8877,
 8343,
 8754,
 15739,
 16280,
 6186,
 10320,
 14224,
 14965,
 13630,
 10384,
 10286,
 14055,
 11892,
 6065,
 6419,
 14224,
 7872,
 9172,
 14452,
 7987,
 9582,
 9139,
 16425,
 14902,
 7500,
 12626,
 15461,
 12606,
 9139]

In [48]:
random_cso_list = [14757,
 6704,
 15019,
 15126,
 8365,
 12070,
 14965,
 6524,
 14886,
 16742,
 9131,
 6119,
 9144,
 15484,
 16516,
 6894,
 13738,
 9560,
 10190,
 7599,
 9242,
 6920,
 17510,
 9060,
 9828,
 15215,
 15005,
 15558,
 12686,
 8548,
 9139,
 7653,
 8653,
 13678,
 15461,
 8754,
 14055,
 10999,
 15334,
 7872,
 9624,
 6577,
 14886,
 14902,
 14797,
 10384,
 10961,
 12052,
 9563,
 6704]

### Inference Methods

In [119]:
csos_descr = {}
for cso in random_cso_list:
    x = get_alerts_by_cso(str(cso))
    descr = ""
    for i in x:
        j = (list(alert_names.index[alert_names['alert_id']==i])[0])
        descr += ""
        descr+= alert_names['alert_identifier'][j].replace("_"," ")
    csos_descr[cso] = descr
len(csos_descr)

48

In [197]:
csos_descr_embed = {}
for cso in random_cso_list:
    x = get_alerts_by_cso(str(cso))
    embed = []
    for i in x:
        j = (list(alert_names.index[alert_names['alert_id']==i])[0])
        descr = alert_names['alert_identifier'][j].replace("_"," ")
        tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(descr, tokenizer_bert)
        list_token_embeddings = np.array(get_bert_embeddings(tokens_tensor, segments_tensors, model_bert))
        descr_embed = np.mean(list_token_embeddings, axis=0)
        embed.append(descr_embed)
        
    embed = np.array(embed)
    cso_embed = np.mean(embed, axis=0)
    csos_descr_embed[cso] = cso_embed

In [207]:
#### Sim
embeddings = get_symptom_embeddings()
nlp_pred_random = {}
for i in random_cso_list:
    nlp_pred_random[i] = remove_cso_from_dict(get_similar_cso_from_description_bert_with_embed(csos_descr_embed[i], embeddings),i)
len(nlp_pred_random)

48

In [122]:
#### Pred
pred_g1_random = {}
for i in random_cso_list:
    pred_g1_random[i] = remove_cso_from_dict(find_rc_for_set(get_alerts_by_cso(str(i)), 10, g), i)
    # except:
        # print(i," skipped")
    
len(pred_g1_random)

48

In [252]:
#### Clust #######
node_cluster_map = nx.get_node_attributes(g, "cluster_id")
cso_cluster_map = {k.split('_')[0]:v for k,v in node_cluster_map.items()}
n_clusters = len(set([v for v in cso_cluster_map.values()]))
rf = pickle.load(open('random_forest.pickle', 'rb'))

combined_cluster_pred_random = {}
for i in random_cso_list:
    df = create_rf_inference_df(get_alerts_by_cso(str(i)), nx.get_node_attributes(g, "alert_id"))
    
    clust1 = find_symptom_cluster_for_set(get_alerts_by_cso(str(i)), 10, g, cso_cluster_map)
    clust2 = rank_cluster(clust1, n_clusters, rf, df)
    
    s = sum([1.0/v for k,v in clust1.items()])
    clust1_1 = {k: 1.0/(v*s) for k,v in clust1.items()}

    clust = {}
    for x in range(n_clusters):
        cl = float(x)
        r1 = clust1_1[cl] if cl in clust1_1 else 0
        r2 = clust2[cl] if cl in clust2 else 0
        clust[cl] = r1+r2
    
    clust = dict(sorted(clust.items(), key=lambda x: x[1], reverse=True)[::])
    
    combined_cluster_pred_random[i] = find_rc_set_bert(clust, 3, cso_cluster_map, csos_descr_embed[i])
    
len(combined_cluster_pred_random)

48

In [182]:
oracle_rc_random, oracle_rem_random = get_oracle_meteor_score(random_cso_list, 'f')

## Results
- take max of ROUGE-l (mode) scores among top z predictions from each method
- average maxes over entire test set to generate average score for each method

#### Extracted vs Extracted

In [272]:
## exracted vs extracted
cols = ['cso_number','comdoublepred_rc','compred_rc','doublepred_rc','pred_rc','nlp_rc','cluster_pred_rc','oracle_rc','comdoublepred_rem','compred_rem','doublepred_rem','pred_rem','nlp_rem','cluster_pred_rem','oracle_rem']
random_cso_rouge_recall_df = pd.DataFrame(columns = cols)
mode = 'f'
z = 5
for i in random_cso_list:
    series = {}
    series['cso_number'] = int(i)
    index = cso_df.index[cso_df['cso_number'] == int(i)].tolist()[0]
    actual_rc = cso_df['root_cause'][index]
    actual_rem = cso_df['remediations'][index]
    
    
    series['nlp_rc'],series['nlp_rem'],a,b  = max_rouge_minus1(nlp_pred_random[i], actual_rc, actual_rem, mode,z)
    series['pred_rc'],series['pred_rem'],a,b = max_rouge_minus1(pred_g1_random[i], actual_rc, actual_rem,mode,z)
    series['compred_rc'],series['compred_rem'],a,b = max_rouge_minus1(compred_g1_random[i], actual_rc, actual_rem,mode,z)
    series['doublepred_rc'],series['doublepred_rem'],a,b = max_rouge_minus1(doublepred_g1_random[i], actual_rc, actual_rem,mode,z)
    series['cluster_pred_rc'],series['cluster_pred_rem'],a,b  = max_rouge_minus1(combined_cluster_pred_random[i], actual_rc, actual_rem, mode,z)
    series['comdoublepred_rc'],series['comdoublepred_rem'],a,b = max_rouge_minus1(comdoublepred_g1_random[i], actual_rc, actual_rem,mode,z)
    
    series['oracle_rc'] = max(remove_cso_from_dict(get_top_n_rc(oracle_rc_random[i],2),i).values())
    series['oracle_rem'] = max(remove_cso_from_dict(get_top_n_rc(oracle_rem_random[i],2),i).values())
    
    random_cso_rouge_recall_df = pd.concat([random_cso_rouge_recall_df, pd.DataFrame([series], columns=cols)], ignore_index=True)
random_cso_rouge_recall_df.head()

,cso_number,comdoublepred_rc,compred_rc,doublepred_rc,pred_rc,nlp_rc,cluster_pred_rc,oracle_rc,comdoublepred_rem,compred_rem,doublepred_rem,pred_rem,nlp_rem,cluster_pred_rem,oracle_rem
0,14757,0.228571,0.228571,0.228571,0.228571,0.214286,0.242424,0.352607,0.148148,0.148148,0.148148,0.148148,0.170213,0.195122,0.211463
1,6704,0.137931,0.173913,0.137931,0.173913,0.181818,0.169492,0.314657,0.185185,0.153846,0.185185,0.153846,0.240000,0.240000,0.220121
2,15019,0.192771,0.192771,0.192771,0.192771,0.238095,0.238095,0.999998,0.150000,0.166667,0.150000,0.166667,0.121212,0.120000,0.162779
3,15126,0.230769,0.230769,0.230769,0.230769,0.312500,0.297872,0.254834,0.256410,0.256410,0.256410,0.256410,0.127660,0.127660,0.177689
4,8365,0.202899,0.225000,0.202899,0.225000,0.160000,0.163265,0.260058,0.160000,0.160000,0.160000,0.160000,0.208333,0.244898,0.254856


In [229]:
show = ['comdoublepred_rc','compred_rc','doublepred_rc','pred_rc','nlp_rc','cluster_pred_rc','oracle_rc','comdoublepred_rem','compred_rem','doublepred_rem','pred_rem','nlp_rem','cluster_pred_rem','oracle_rem']

In [273]:
random_cso_rouge_recall_df[show].mean()

comdoublepred_rc     0.198905
compred_rc           0.200542
doublepred_rc        0.198749
pred_rc              0.202325
nlp_rc               0.211401
cluster_pred_rc      0.242112
oracle_rc            0.402400
comdoublepred_rem    0.155980
compred_rem          0.154367
doublepred_rem       0.159314
pred_rem             0.157894
nlp_rem              0.177013
cluster_pred_rem     0.219413
oracle_rem           0.200598
dtype: float64

In [185]:
random_cso_rouge_recall_df[show].max()

comdoublepred_rc     0.270766
compred_rc           0.285671
doublepred_rc        0.270766
pred_rc              0.285671
nlp_rc               0.310361
cluster_pred_rc      0.310361
oracle_rc            0.999998
comdoublepred_rem    0.262084
compred_rem          0.262084
doublepred_rem       0.262084
pred_rem             0.262084
nlp_rem              0.312373
cluster_pred_rem     0.233796
oracle_rem           0.349578
dtype: float64